8.1 Using the code presented in Section 8.6:  
(a) Generate a dataset (X, y)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

def get_test_data(n_features=20, n_informative=5, n_redundant=5, n_samples=3000):
    from sklearn.datasets import make_classification
    X, cont=make_classification(n_samples=n_samples, n_features=n_features, n_informative=n_informative, n_redundant=n_redundant, random_state=0, shuffle=False)
    df0=pd.date_range(periods=n_samples, freq='1D', end=datetime.today())
    X, cont=pd.DataFrame(X, index=df0), pd.Series(cont, index=df0).to_frame('bin')
    df0=[f'I_{str(i)}' for i in range(n_informative)]+[f'R_{str(i)}' for i in range(n_informative, n_informative+n_redundant)]+[f'N_{str(i)}' for i in range(n_informative+n_redundant, n_features)]
    X.columns=df0
    cont['w']=1./cont.shape[0]
    cont['t1']=pd.Series(cont.index, index=cont.index)
    return X, cont

test_data, cont=get_test_data()
print(test_data.head())
print(cont.head())

                                 I_0       I_1       I_2       I_3       I_4  \
2017-02-25 10:56:26.285645 -0.737441  1.480017 -1.624384 -0.907109  1.852716   
2017-02-26 10:56:26.285645 -0.722167  2.694209 -0.848284  1.394876 -0.676037   
2017-02-27 10:56:26.285645 -2.464807  1.849071  0.021696  3.202498  0.194839   
2017-02-28 10:56:26.285645  1.557030  1.290159 -1.044263 -1.673964  2.077268   
2017-03-01 10:56:26.285645 -0.775799  1.620273 -2.026289  0.775127  0.127839   

                                 R_5       R_6       R_7       R_8       R_9  \
2017-02-25 10:56:26.285645 -1.042550  0.206819 -0.282577  0.667836 -1.155404   
2017-02-26 10:56:26.285645 -1.058662  0.937675  0.242056  0.792361  1.656174   
2017-02-27 10:56:26.285645 -2.238507 -0.619387 -1.045632  3.676729 -0.283826   
2017-02-28 10:56:26.285645  0.172160 -1.028541  1.640022 -1.346945 -0.755371   
2017-03-01 10:56:26.285645 -1.718214  0.983768 -0.961620  1.657738 -0.341892   

                                N_10  

(b) Apply a PCA transformationon X, which we denotė X.

In [2]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def get_orthogonal_features(X:pd.DataFrame, threshold:float=0.95)->pd.DataFrame:
    scaler=StandardScaler()
    Z=scaler.fit_transform(X)
    pca=PCA(n_components=threshold)
    princial_components=pca.fit_transform(Z)
    princial_components=pd.DataFrame(princial_components, index=X.index, columns=[f'PC_{i+1}' for i in range(princial_components.shape[1])])
    return princial_components

pca_test_data=get_orthogonal_features(test_data, 0.95)

(c) Compute MDI, MDA, and SFI feature importance on 
(X_pca,y), where the base estimator is RF.

In [3]:
from itertools import product
from utils import PurgedKFold, cv_score

def get_mdi_importance(fit, feature_names):
    df0={i:tree.feature_importances_ for i,tree in enumerate(fit.estimators_)}
    df0=pd.DataFrame.from_dict(df0, orient='index')
    df0.columns=feature_names
    df0=df0.replace(0, np.nan) # zero importance가 average되는 것을 방지
    imp=pd.concat({'mean':df0.mean(axis=0).rename('mean'), 'std':df0.std(axis=0).rename('std')/np.sqrt(len(df0))}, axis=1)
    imp/=imp['mean'].sum() # normalize  
    return imp

def get_mda_importance(clf, X, y, cv, sample_weight:pd.Series, t1:pd.Series, pct_embargo:float, scoring='neg_log_loss'):
    # get feature importance based on OOS score reduction
    if scoring not in ['neg_log_loss', 'accuracy']:
        raise ValueError(f"scoring must be 'neg_log_loss' or 'accuracy', got {scoring}")
    from sklearn.metrics import log_loss, accuracy_score
    cv_gen=PurgedKFold(n_splits=cv, t1=t1, pct_embargo=pct_embargo)
    scr0, scr1=pd.Series(), pd.DataFrame(columns=X.columns)
    for i, (train_idx, test_idx) in enumerate(cv_gen.split(X, y)):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
        sample_weight_train=sample_weight.iloc[train_idx]
        sample_weight_test=sample_weight.iloc[test_idx]
        clf.fit(X_train, y_train, sample_weight=sample_weight_train)
        if scoring=='neg_log_loss':
            scr0.loc[i]=-log_loss(y_test, clf.predict_proba(X_test), sample_weight=sample_weight_test.values, labels=clf.classes_)
        else:
            scr0.loc[i]=accuracy_score(y_test, clf.predict(X_test), sample_weight=sample_weight_test.values)
        for j in X.columns:
            # inplace 전체 copy 제거 + numpy array shuffle
            shuffled_col = X_test[j].values.copy()
            np.random.shuffle(shuffled_col)
            X1_ = X_test.copy()
            X1_[j] = shuffled_col
            if scoring=='neg_log_loss':
                scr1.loc[i,j]=-log_loss(y_test, clf.predict_proba(X1_), sample_weight=sample_weight_test.values, labels=clf.classes_)
            else:
                scr1.loc[i,j]=accuracy_score(y_test, clf.predict(X1_), sample_weight=sample_weight_test.values)
    imp=(-scr1).add(scr0, axis=0)
    if scoring=='neg_log_loss':
        imp=imp/-scr1 # (original - permuted) / permuted
    else:
        imp=imp/(1.0-scr1)
    imp=pd.concat({'mean':imp.mean(axis=0).rename('mean'), 'std':imp.std(axis=0).rename('std')/np.sqrt(len(imp))}, axis=1)
    return imp, scr0.mean()

from utils import cv_score

def get_sfi(feature_names, clf, normalizd_X, y:pd.Series,sample_weight:pd.Series, scoring, cv_gen)->pd.DataFrame:
    imp=pd.DataFrame(columns=['mean', 'std'])
    for feature in feature_names:
        df0=cv_score(clf, X=normalizd_X[[feature]], y=y, sample_weight=sample_weight, scoring=scoring, cv_gen=cv_gen)
        imp.loc[feature, 'mean']=df0.mean(axis=0)
        imp.loc[feature, 'std']=df0.std(axis=0)/np.sqrt(len(df0))
    return imp

def get_feature_importance(trans_x, cont, n_estimators=1000, cv=10, max_samples=1., num_threads=24, pct_embargo=0.1, scoring='accuracy', method='SFI', min_weight_leaf=0.05, **kwargs):
    '''
    Random Forest Feature Importance
    '''
    from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

    clf=RandomForestClassifier(n_estimators=1, criterion='entropy', bootstrap=False, class_weight='balanced_subsample', min_weight_fraction_leaf=min_weight_leaf)
    clf=BaggingClassifier(estimator=clf, n_estimators=n_estimators, max_features=1., max_samples=max_samples, oob_score=True)
    fit=clf.fit(X=trans_x, y=cont['bin'], sample_weight=cont['w'].values)
    oob_score=fit.oob_score_
    if method=='MDI':
        imp=get_mdi_importance(fit, feature_names=trans_x.columns)
        oos=cv_score(clf, X=trans_x, y=cont['bin'], sample_weight=cont['w'], scoring=scoring, cv_gen=PurgedKFold(n_splits=cv, t1=cont['t1'], pct_embargo=pct_embargo)).mean()
    elif method=='MDA':
        imp, oos=get_mda_importance(clf, X=trans_x, y=cont['bin'], cv=cv, sample_weight=cont['w'], t1=cont['t1'], pct_embargo=pct_embargo, scoring=scoring)
    elif method=='SFI':
        cv_gen=PurgedKFold(n_splits=cv, t1=cont['t1'], pct_embargo=pct_embargo)
        oos=cv_score(clf, X=trans_x, y=cont['bin'], sample_weight=cont['w'], scoring=scoring, cv_gen=cv_gen).mean()
        imp=get_sfi(trans_x.columns, clf, trans_x, y=cont['bin'], sample_weight=cont['w'], scoring=scoring, cv_gen=cv_gen)
    return imp, oob_score, oos

def test_feature_importance(trans_x, cont, n_features=40, n_informative=10, n_redundant=10, n_estimators=1000, n_samples=10000, cv=10):
    dict0={'min_weight_leaf':[0.], 'scoring':['accuracy'], 'method':['MDA', 'SFI', 'MDI'], 'max_samples':[1.]}
    jobs, out=(dict(zip(dict0.keys(), values)) for values in product(*dict0.values())), []
    kargs={'n_estimators':n_estimators, 'tag':'test_func', 'cv': cv}

    def plot_feature_importance(imp, oob, oos, method, scoring, **kargs):
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, imp.shape[0]/5))
        imp=imp.sort_values('mean', ascending=True)
        ax=imp['mean'].plot(xerr=imp['std'], error_kw={'ecolor':'r'}, color='b', kind='barh')    
        if method=='MDI':
            plt.xlim([0, imp.sum(axis=1).max()])
            plt.axvline(1/len(imp), color='r', linestyle='--')
        for i, j in zip(ax.patches, imp.index):
            ax.text(i.get_width()/2, i.get_y()+i.get_height()/2, j, ha='center', va='center', fontsize=10)
        plt.title(f"{method} Feature Importance\nOOB: {oob:.4f}, OOS: {oos:.4f}", fontsize=15)
        plt.savefig(f"feature_importance_{method}_{scoring}_PCA.png", dpi=300)
        plt.clf()
        plt.close()
        
    out={}
    for job in jobs:
        imp, oob, oos=get_feature_importance(trans_x=trans_x,cont=cont, **job)
        kargs.update(job)
        plot_feature_importance(imp, oob, oos, **kargs)
        out[job['method']]=imp
    return out

test_feature_importance(pca_test_data, cont, n_features=20, n_informative=5, n_redundant=5, n_estimators=400, n_samples=3000, cv=5)


/Users/kimjuho/Desktop/quant/AFML/chapter8/utils.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_t1_idx = self.t1.index.searchsorted(self.t1[test_indices].max())
/Users/kimjuho/Desktop/quant/AFML/chapter8/utils.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_t1_idx = self.t1.index.searchsorted(self.t1[test_indices].max())
/Users/kimjuho/Desktop/quant/AFML/chapter8/utils.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[

{'MDA':            mean       std
 PC_1   0.331518   0.07642
 PC_2    0.15529  0.073922
 PC_3   0.393998  0.108257
 PC_4    0.36803  0.105138
 PC_5   0.019391  0.023959
 PC_6   0.017223  0.044161
 PC_7   0.063516    0.0341
 PC_8   0.028834  0.029003
 PC_9   0.011073  0.023729
 PC_10  0.047615  0.036887
 PC_11 -0.024412  0.020877
 PC_12  0.035086  0.018147
 PC_13  0.028798  0.020298
 PC_14  0.016378  0.010761,
 'SFI':            mean       std
 PC_1       0.45  0.030858
 PC_2       0.44  0.027146
 PC_3      0.482  0.018916
 PC_4   0.653333  0.028891
 PC_5   0.423333  0.019408
 PC_6   0.441333  0.017728
 PC_7   0.427667  0.017506
 PC_8   0.444667  0.019317
 PC_9   0.435667  0.021644
 PC_10     0.444  0.023976
 PC_11  0.424667  0.013484
 PC_12     0.453    0.0197
 PC_13  0.443333  0.020363
 PC_14  0.432333  0.011672,
 'MDI':            mean       std
 PC_1   0.133860  0.000843
 PC_2   0.108319  0.000677
 PC_3   0.143617  0.000678
 PC_4   0.309973  0.001178
 PC_5   0.029697  0.000271
 PC_6

8.2 From exercise 1, generate a new dataset (̈
X,y), wherë
X_new is a feature union of X anḋ X_pca.
(a) Compute MDI, MDA, and SFI feature importance on (̈
X,y), where the base
estimator is RF.

In [4]:
X_new=pd.concat([test_data, pca_test_data], axis=1)

def test_feature_importance(trans_x, cont, n_features=40, n_informative=10, n_redundant=10, n_estimators=1000, n_samples=10000, cv=10):
    dict0={'min_weight_leaf':[0.], 'scoring':['accuracy'], 'method':['MDA', 'SFI', 'MDI'], 'max_samples':[1.]}
    jobs, out=(dict(zip(dict0.keys(), values)) for values in product(*dict0.values())), []
    kargs={'n_estimators':n_estimators, 'tag':'test_func', 'cv': cv}

    def plot_feature_importance(imp, oob, oos, method, scoring, **kargs):
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, imp.shape[0]/5))
        imp=imp.sort_values('mean', ascending=True)
        ax=imp['mean'].plot(xerr=imp['std'], error_kw={'ecolor':'r'}, color='b', kind='barh')    
        if method=='MDI':
            plt.xlim([0, imp.sum(axis=1).max()])
            plt.axvline(1/len(imp), color='r', linestyle='--')
        for i, j in zip(ax.patches, imp.index):
            ax.text(i.get_width()/2, i.get_y()+i.get_height()/2, j, ha='center', va='center', fontsize=10)
        plt.title(f"{method} Feature Importance\nOOB: {oob:.4f}, OOS: {oos:.4f}", fontsize=15)
        plt.savefig(f"feature_importance_{method}_{scoring}_PCA_comb.png", dpi=300)
        plt.clf()
        plt.close()
        
    out={}
    for job in jobs:
        imp, oob, oos=get_feature_importance(trans_x=trans_x,cont=cont, **job)
        kargs.update(job)
        plot_feature_importance(imp, oob, oos, **kargs)
        out[job['method']]=imp
    return out

feature_imps=test_feature_importance(X_new, cont, n_features=40, n_informative=10, n_redundant=10, n_estimators=1000, n_samples=10000, cv=10)

/Users/kimjuho/Desktop/quant/AFML/chapter8/utils.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_t1_idx = self.t1.index.searchsorted(self.t1[test_indices].max())
/Users/kimjuho/Desktop/quant/AFML/chapter8/utils.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_t1_idx = self.t1.index.searchsorted(self.t1[test_indices].max())
/Users/kimjuho/Desktop/quant/AFML/chapter8/utils.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[

8.3 Take the results from exercise 2:  
(a) Drop the most important features according to each method, resulting in a  
features matrix X_new2  

In [5]:
important_features=[]
for method, imp in feature_imps.items():
    cur_imp=imp['mean'].sort_values(ascending=False).index.values[0]
    important_features.append(cur_imp)
X_new2=X_new.drop(columns=important_features, axis=1)


(b) Compute MDI, MDA, and SFI feature importance on (  
X_new2,y), where the base   
estimator is RF.    

In [6]:
def test_feature_importance(trans_x, cont, n_features=40, n_informative=10, n_redundant=10, n_estimators=1000, n_samples=10000, cv=10):
    dict0={'min_weight_leaf':[0.], 'scoring':['accuracy'], 'method':['MDA', 'SFI', 'MDI'], 'max_samples':[1.]}
    jobs, out=(dict(zip(dict0.keys(), values)) for values in product(*dict0.values())), []
    kargs={'n_estimators':n_estimators, 'tag':'test_func', 'cv': cv}

    def plot_feature_importance(imp, oob, oos, method, scoring, **kargs):
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, imp.shape[0]/5))
        imp=imp.sort_values('mean', ascending=True)
        ax=imp['mean'].plot(xerr=imp['std'], error_kw={'ecolor':'r'}, color='b', kind='barh')    
        if method=='MDI':
            plt.xlim([0, imp.sum(axis=1).max()])
            plt.axvline(1/len(imp), color='r', linestyle='--')
        for i, j in zip(ax.patches, imp.index):
            ax.text(i.get_width()/2, i.get_y()+i.get_height()/2, j, ha='center', va='center', fontsize=10)
        plt.title(f"{method} Feature Importance\nOOB: {oob:.4f}, OOS: {oos:.4f}", fontsize=15)
        plt.savefig(f"feature_importance_{method}_{scoring}_PCA_comb2.png", dpi=300)
        plt.clf()
        plt.close()
        
    out={}
    for job in jobs:
        imp, oob, oos=get_feature_importance(trans_x=trans_x,cont=cont, **job)
        kargs.update(job)
        plot_feature_importance(imp, oob, oos, **kargs)
        out[job['method']]=imp
    return out


feature_imps=test_feature_importance(X_new2,  cont, n_features=40, n_informative=10, n_redundant=10, n_estimators=1000, n_samples=10000, cv=10)

/Users/kimjuho/Desktop/quant/AFML/chapter8/utils.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_t1_idx = self.t1.index.searchsorted(self.t1[test_indices].max())
/Users/kimjuho/Desktop/quant/AFML/chapter8/utils.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_t1_idx = self.t1.index.searchsorted(self.t1[test_indices].max())
/Users/kimjuho/Desktop/quant/AFML/chapter8/utils.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[